In [1]:
import pandas as pd


In [8]:
# Import mapstreet data 

mapstreet = pd.read_csv("src_data\GISMAPSTREET.txt", sep=" ", header=None, skip_blank_lines=True,
names=["MINORGRIDNAME", "STREETNAME", "ROADDIRECTIONPREFIXCD", "ROADNAME", "ROADTYPESUFFIXCD", "ROADDIRECTIONSUFFIXCD", "ZIPCD", "RIGHTZIPCD", "LOWADDRESSRANGEVALUE", "HIGHADDRESSRANGEVALUE", "TOWNSHIPNAME", "MUNICIPALITYNAME", "NUMCENTERLINES", "INTERSECTINGLENGTH", "GEOM_WKT"])

# mapstreet.dropna(inplace=True)
# mapstreet.drop_duplicates(keep='first', inplace=True)
mapstreet.drop(columns=["ROADDIRECTIONPREFIXCD", "ROADNAME", "ROADTYPESUFFIXCD", "ROADDIRECTIONSUFFIXCD", "RIGHTZIPCD", "LOWADDRESSRANGEVALUE", "HIGHADDRESSRANGEVALUE", "TOWNSHIPNAME"], inplace=True)

dropped_cols = mapstreet.drop(columns=["ROADDIRECTIONPREFIXCD", "ROADNAME", "ROADTYPESUFFIXCD", "ROADDIRECTIONSUFFIXCD", "RIGHTZIPCD", "LOWADDRESSRANGEVALUE", "HIGHADDRESSRANGEVALUE", "TOWNSHIPNAME"])

# mapstreet.fillna("", inplace=True)

print(dropped_cols)
# print(mapstreet)
# mapstreet.dtypes


                                            MINORGRIDNAME  \
0       MINORGRIDNAME|STREETNAME|ROADDIRECTIONPREFIXCD...   
1                                             AB12B|117TH   
2                                             AB12B|118TH   
3                                             AB12B|119TH   
4                                             AB12B|120TH   
...                                                   ...   
168648                                       YN31D|COUNTY   
168649                                            YN31D|E   
168650                                            YN31D|E   
168651                                       YN32A|COUNTY   
168652                                            YN32A|S   

                                               STREETNAME               ZIPCD  \
0                                                     NaN                 NaN   
1       PL||117TH|PL||46394|46394|618|671|NORTH|HAMMON...   -87.5121713044794   
2       ST||118TH|ST||46

In [3]:
# We can only map lines with WKT coordinates. Drop all rows which have no value in GEO_WKT
mapstreet.dropna(subset=['GEOM_WKT'], how='all', inplace=True)
print(mapstreet)
mapstreet.to_csv(r'./test_data/mapstreet-test.csv', header=True, index=None)

                                            MINORGRIDNAME  \
1                                             AB12B|117TH   
2                                             AB12B|118TH   
5       AB12B|ALLEY||ALLEY|||||||NORTH|HAMMOND|27|6933...   
6                                             AB12B|BROWN   
7                                            AB12B|BURTON   
...                                                   ...   
168646                                            YN31C|E   
168647                                            YN31C|E   
168648                                       YN31D|COUNTY   
168649                                            YN31D|E   
168652                                            YN32A|S   

                                               STREETNAME  \
1       PL||117TH|PL||46394|46394|618|671|NORTH|HAMMON...   
2       ST||118TH|ST||46394|46394|638|751|NORTH|HAMMON...   
5                                     ((-87.5116768034287   
6       AVE||BROWN|AVE|

In [ ]:
consequence = pd.read_csv("src_data\Street Consequence - Sheet1.csv", sep=",", header=None, skip_blank_lines=True,
names=["STATE", "MUNICIPALITY", "STREETNAME", "MINORGRIDNAME", "CONSEQUENCE"])

consequence.dropna(inplace=True)
print(consequence)
consequence.dtypes

In [ ]:
merged_df = pd.merge(consequence, mapstreet, on="STREETNAME")

# merged_df.dropna(axis=1, inplace=True)
# merged_df.dropna(axis=0, inplace=True)
merged_df.drop(columns=["MINORGRIDNAME_y", "ROADDIRECTIONPREFIXCD", "ROADNAME", "ROADTYPESUFFIXCD", "ROADDIRECTIONSUFFIXCD", "LEFTZIPCD", "RIGHTZIPCD", "LOWADDRESSRANGEVALUE", "HIGHADDRESSRANGEVALUE", "TOWNSHIPNAME", "MUNICIPALITYNAME"], inplace=True)
merged_df.rename(columns={"MINORGRIDNAME_x" : "MINORGRIDNAME"}, inplace=True)

print(merged_df)
merged_df.dtypes

In [ ]:
merged_df.to_csv(r'./combined_test.csv', header=True)